In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\leers\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\leers\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents_test.json').read()
intents = json.loads(data_file)

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [11]:
# intents
# intents['intents']

In [12]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [16]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [33]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax


model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2134 - accuracy: 0.1204
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1007 - accuracy: 0.2871
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0953 - accuracy: 0.2367
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9627 - accuracy: 0.2930
Epoch 5/200
10/10 [==============================] - 0s 4ms/step - loss: 1.8330 - accuracy: 0.4287
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 1.8415 - accuracy: 0.4612
Epoch 7/200
10/10 [==============================] - 0s 4ms/step - loss: 1.6646 - accuracy: 0.5559
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.5934 - accuracy: 0.5641
Epoch 9/200
10/10 [==============================] - 0s 4ms/step - loss: 1.4476 - accuracy: 0.5576
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.3830 - accuracy: 0.5432
Epoch 11/

10/10 [==============================] - 0s 3ms/step - loss: 0.0370 - accuracy: 1.0000
Epoch 83/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0281 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0676 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0206 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0182 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0761 - accuracy: 0.9447
Epoch 88/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0573 - accuracy: 0.9862
Epoch 89/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0333 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0205 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0217 - accuracy: 1.0000
Epoch 92/200


10/10 [==============================] - 0s 3ms/step - loss: 0.0084 - accuracy: 1.0000
Epoch 164/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0141 - accuracy: 0.9961
Epoch 165/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0104 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0303 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0176 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0302 - accuracy: 0.9961
Epoch 169/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0188 - accuracy: 0.9892
Epoch 170/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0130 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0152 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0253 - accuracy: 1.0000
Epoc